# Machine Learning Methods for Predicting Mental Health Outcomes 

### Team Members 

Humaira Nasir (humairan)

Kate Wasmer (kwasmer)

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

## Part I: Preprocessing 

### I. Data Cleaning & Handling 

Figure out countries that are in one data set but not the other to improve merging and filtering. 

In [2]:
happiness = pd.read_csv("world-happiness-modified.csv")
happiness_unique_countries = set(happiness.country)

In [3]:
mental_health = pd.read_csv("mental-health-modified.csv", low_memory = False)
mental_health = mental_health.drop(columns = "Unnamed: 0")
unique_countries_mh = set(mental_health.country) # get unique country names 

In [4]:
# Countries that are in mental health dataset but not world happiness 
difference_A = unique_countries_mh.difference(happiness_unique_countries)
for elem in difference_A:
    print(elem)

Eastern Sub-Saharan Africa
Congo
American Samoa
Cote d'Ivoire
Tropical Latin America
High-income
Barbados
Equatorial Guinea
High SDI
Guinea-Bissau
Cape Verde
Sao Tome and Principe
Sub-Saharan Africa
North America
South Sudan
Guam
Tonga
Central Europe, Eastern Europe, and Central Asia
Greenland
Cuba
Central Asia
Low-middle SDI
Democratic Republic of Congo
Puerto Rico
Southern Latin America
Timor
Northern Mariana Islands
East Asia
Low SDI
Central Latin America
World
Northern Ireland
Seychelles
Central Europe
Kiribati
Solomon Islands
England
South Asia
Andean Latin America
Somalia
Taiwan
Saint Lucia
Andorra
Caribbean
Eritrea
Antigua and Barbuda
Dominica
Vanuatu
Southern Sub-Saharan Africa
Southeast Asia
United States Virgin Islands
Papua New Guinea
China
Grenada
Australasia
Macedonia
Western Sub-Saharan Africa
North Africa and Middle East
High-income Asia Pacific
Latin America and Caribbean
Maldives
North Korea
Palestine
Micronesia (country)
Western Europe
Brunei
Fiji
Southeast Asia, East

In [5]:
# Countries that are in happiness data set but not mental health 
difference_B = happiness_unique_countries.difference(unique_countries_mh)
for elem in difference_B:
    print(elem)

Taiwan Province of China
Congo (Kinshasa)
Congo (Brazzaville)
Ivory Coast
Palestinian Territories
North Macedonia


In [6]:
# Replacing the index of Macedonia was written with the help of Copilot. 
mental_health.replace({"Macedonia":"North Macedonia"},inplace=True)

In [7]:
happiness.replace({ "Palestinian Territories": "Palestine",
                    "Congo (Kinshasa)": "Democratic Republic of the Congo",
                    "Taiwan Province of China": "Taiwan",
                    "Ivory Coast": "Cote d'Ivoire",
                    "Congo (Brazzaville)": "Congo"},
                    inplace=True)

In [8]:
# Drop countries that are not in the happiness data set for easier merging. 
mental_health = mental_health[mental_health.country\
                              .isin(happiness.country)] # Generated w/Copilot
mental_health

,country,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260
...,...,...,...,...,...,...,...,...,...
6463,Zimbabwe,2013,0.155670,0.607993,0.117248,3.090168,0.766280,3.128192,1.515641
6464,Zimbabwe,2014,0.155993,0.608610,0.118073,3.093964,0.768914,3.140290,1.515470
6465,Zimbabwe,2015,0.156465,0.609363,0.119470,3.098687,0.771802,3.155710,1.514751
6466,Zimbabwe,2016,0.157111,0.610234,0.121456,3.104294,0.772275,3.174134,1.513269


Merging on right was decided on after experimenting with the left, outer, and inner methods. 

In [9]:
data_merged = mental_health.merge(happiness, on=["country", "year"], how="right")
data_merged.head()

,country,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders,Unnamed: 0,happiness,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption
0,Afghanistan,2008,0.164639,0.704480,0.093589,4.860437,2.483862,4.129656,0.659501,0,3.724,7.370,0.451,50.80,0.718,0.168,0.882
1,Afghanistan,2009,0.164932,0.704925,0.095166,4.861533,2.543884,4.129972,0.661185,1,4.402,7.540,0.552,51.20,0.679,0.190,0.850
2,Afghanistan,2010,0.165130,0.705313,0.097327,4.862777,2.571349,4.130874,0.662062,2,4.758,7.647,0.539,51.60,0.600,0.121,0.707
3,Afghanistan,2011,0.165272,0.705688,0.098638,4.864773,2.573170,4.130862,0.662254,3,3.832,7.620,0.521,51.92,0.496,0.162,0.731
4,Afghanistan,2012,0.165424,0.706086,0.099891,4.867283,2.576189,4.132485,0.662372,4,3.783,7.705,0.521,52.24,0.531,0.236,0.776


In [10]:
data_merged.isnull().sum()

country                    0
year                       0
schizophrenia            341
bipolar_disorder         341
eating_disorders         341
anxiety_disorders        341
drug_use_disorders       341
depression               341
alcohol_use_disorders    341
Unnamed: 0                 0
happiness                  0
gdp_per_capita             0
social_support             0
life_expectancy            0
freedom                    0
generosity                 0
corruption                 0
dtype: int64

In [11]:
# Drop dummy column that was created with the merge. 
data_merged = data_merged.drop(columns="Unnamed: 0")

In [12]:
# Perform imputation on the NaN values (all are disorders).
disorders_subset = data_merged[["alcohol_use_disorders", "schizophrenia", 
                                "bipolar_disorder", "anxiety_disorders", 
                                "eating_disorders", "depression", 
                                "drug_use_disorders"]]
disorders_labels = disorders_subset.columns

In [13]:
# Subset remaining data that we don't want to impute 
data_other = data_merged.drop(columns=disorders_labels)
data_other.head()

,country,year,happiness,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776


In [14]:
num_imputer = SimpleImputer(strategy="median")
imputed = num_imputer.fit_transform(disorders_subset)
data_num_imputed = pd.DataFrame(imputed, columns=disorders_labels)

In [15]:
# Concatenate the two subsetted data frames back together. 
data_imputed = pd.concat([data_other, data_num_imputed], axis=1)

In [16]:
# Verify that all null values were imputed. 
data_imputed.isnull().sum()

country                  0
year                     0
happiness                0
gdp_per_capita           0
social_support           0
life_expectancy          0
freedom                  0
generosity               0
corruption               0
alcohol_use_disorders    0
schizophrenia            0
bipolar_disorder         0
anxiety_disorders        0
eating_disorders         0
depression               0
drug_use_disorders       0
dtype: int64

In [17]:
# View descriptive statistics to get an idea of how to create labels for
# mental health outcomes. 
data_imputed[disorders_labels].describe()

,alcohol_use_disorders,schizophrenia,bipolar_disorder,anxiety_disorders,eating_disorders,depression,drug_use_disorders
count,1712.000000,1712.000000,1712.000000,1712.000000,1712.000000,1712.000000,1712.000000
mean,1.673125,0.208194,0.734953,3.865152,0.249098,3.453111,0.805390
std,0.897182,0.039279,0.132691,1.151859,0.155949,0.588234,0.434205
min,0.457413,0.148279,0.352298,2.024573,0.084403,2.194091,0.416284
25%,1.130166,0.188419,0.641011,3.070240,0.147816,3.085895,0.559996
50%,1.532773,0.199308,0.707686,3.475837,0.200172,3.461686,0.673695
75%,1.856625,0.211556,0.787909,4.300798,0.264483,3.734583,0.885406
max,5.474668,0.375087,1.206597,8.908704,0.943991,5.679566,3.452476


In [18]:
def get_quantile_score(data: pd.DataFrame, disorder: str, disorder_rate: float) -> int: 
    """This function returns a score from 1-4, based on the quantile that the 
    disorder rate falls into. These scores will later be summed to create an 
    overall mental health score, which will then be used to create categorical
    labels for the mental health outcomes.
    
    (This prompt was written with the help of Copilot.)

    Keyword arguments:
    data -- the data frame that contains the mental health data
    disorder -- the name of the disorder that we want to obtain the quantile score for
    disorder_rate -- the rate of the disorder in a specific row (country-year pair)
    """
    if (disorder_rate < data[disorder].quantile(0.25)):
        return 1 
    elif (disorder_rate < data[disorder].median()):
        return 2
    elif (disorder_rate < data[disorder].quantile(0.75)):
        return 3 
    else:
        return 4

In [19]:
# Note: Lambda function was written with the aid of Copilot. 
score_columns = [] # Save column names for later. 
for disorder in disorders_labels: 
    data_imputed[disorder + "_score"] = data_imputed[disorder].\
        apply(lambda x: get_quantile_score(data_imputed, disorder, x))
    score_columns.append(disorder + "_score")

In [20]:
data_imputed["total_score"] = data_imputed[score_columns].sum(axis=1)

In [21]:
data_imputed["total_score"].describe()

count    1712.000000
mean       18.199182
std         4.930438
min         7.000000
25%        14.000000
50%        19.000000
75%        21.000000
max        28.000000
Name: total_score, dtype: float64

In [22]:
def get_final_category(score: int) -> str:
    """This function returns a categorical label based on the total mental health
    score of a country-year pair. (This prompt was written with the help of Copilot.)

    Keyword arguments:
    score -- the total mental health score of a country-year pair
    """
    total_col = data_imputed["total_score"]
    if (score < total_col.quantile(0.25)):
        return "Low"
    elif (score < total_col.median()):
        return "Medium Low"
    elif (score < total_col.quantile(0.75)):
        return "Medium High"
    else:
        return "High"

In [23]:
data_imputed["mental_health_prevalence"] = data_imputed["total_score"].\
    apply(get_final_category)

In [24]:
# Verify that the new column was created successfully.
data_imputed.head()

,country,year,happiness,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption,alcohol_use_disorders,...,drug_use_disorders,alcohol_use_disorders_score,schizophrenia_score,bipolar_disorder_score,anxiety_disorders_score,eating_disorders_score,depression_score,drug_use_disorders_score,total_score,mental_health_prevalence
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.659501,...,2.483862,1,1,2,4,1,4,4,17,Medium Low
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.661185,...,2.543884,1,1,2,4,1,4,4,17,Medium Low
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.662062,...,2.571349,1,1,2,4,1,4,4,17,Medium Low
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.662254,...,2.573170,1,1,2,4,1,4,4,17,Medium Low
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.662372,...,2.576189,1,1,2,4,1,4,4,17,Medium Low


In [25]:
# Drop unnecessary variables.
data_imputed = data_imputed.drop(columns = score_columns)
data_imputed = data_imputed.drop(columns = "total_score")
data_imputed.head()

,country,year,happiness,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption,alcohol_use_disorders,schizophrenia,bipolar_disorder,anxiety_disorders,eating_disorders,depression,drug_use_disorders,mental_health_prevalence
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.659501,0.164639,0.704480,4.860437,0.093589,4.129656,2.483862,Medium Low
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.661185,0.164932,0.704925,4.861533,0.095166,4.129972,2.543884,Medium Low
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.662062,0.165130,0.705313,4.862777,0.097327,4.130874,2.571349,Medium Low
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.662254,0.165272,0.705688,4.864773,0.098638,4.130862,2.573170,Medium Low
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.662372,0.165424,0.706086,4.867283,0.099891,4.132485,2.576189,Medium Low


In [26]:
# Split predictors and target variable for supervised learning. 
X = data_imputed.drop(columns="mental_health_prevalence")
y = data_imputed["mental_health_prevalence"]
X.drop(columns=["country","year","alcohol_use_disorders","schizophrenia","bipolar_disorder","anxiety_disorders","eating_disorders","depression","drug_use_disorders"],inplace=True)

In [27]:
X.head()

,happiness,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption
0,3.724,7.370,0.451,50.80,0.718,0.168,0.882
1,4.402,7.540,0.552,51.20,0.679,0.190,0.850
2,4.758,7.647,0.539,51.60,0.600,0.121,0.707
3,3.832,7.620,0.521,51.92,0.496,0.162,0.731
4,3.783,7.705,0.521,52.24,0.531,0.236,0.776


### II. Feature scaling & normalization  

In [28]:
# Split data into training and testing sets (80% training, 20% testing).
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Filter out numerical and categorical columns for proper encoding.
X_num = X.select_dtypes(include="number")
X_cat = X.select_dtypes(exclude="number")

In [30]:
num_attribs = list(X_num.columns)
cat_attribs = list(X_cat.columns)
all_attribs = list(X_train.columns)

In [31]:
combined_pipeline = ColumnTransformer([
    ("num", StandardScaler(), num_attribs),
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), cat_attribs),
]) 
data_processed = combined_pipeline.fit_transform(X_train)
test_data_processed = combined_pipeline.transform(X_test)

In [32]:
data_processed

array([[ 1.12869036,  1.02169755,  0.982164  , ...,  0.77823595,
        -1.23028648,  0.79518349],
       [ 0.60915417,  1.5594777 ,  0.45203981, ...,  0.25637614,
        -0.47728154, -3.78589062],
       [ 1.36780008,  1.15465112,  1.01478702, ...,  0.65834923,
        -0.20570599, -0.42820162],
       ...,
       [-0.3692617 ,  0.476247  , -0.7305449 , ..., -0.4347355 ,
         0.43619986,  1.05468943],
       [ 1.79679105,  1.25351403,  1.30023851, ...,  1.16610472,
         0.85590753, -2.63135402],
       [-1.5076701 , -2.07202976, -1.0893982 , ...,  0.07301999,
         0.45471637,  0.02725777]])

In [33]:
# Verify that the shape of the array is equal to the shape of the training data
assert(data_processed.shape == X_train.shape)

### III. Dimensionality Reduction 

In [34]:
# Check for highly correlated variables -- we want to avoid collinearity. 
matrix = X_num.corr()
print(matrix)

                 happiness  gdp_per_capita  social_support  life_expectancy  \
happiness         1.000000        0.793122        0.713182         0.754790   
gdp_per_capita    0.793122        1.000000        0.706319         0.860163   
social_support    0.713182        0.706319        1.000000         0.616918   
life_expectancy   0.754790        0.860163        0.616918         1.000000   
freedom           0.523947        0.351963        0.410219         0.384190   
generosity        0.182087       -0.025106        0.055560         0.017870   
corruption       -0.448488       -0.343940       -0.227278        -0.335278   

                  freedom  generosity  corruption  
happiness        0.523947    0.182087   -0.448488  
gdp_per_capita   0.351963   -0.025106   -0.343940  
social_support   0.410219    0.055560   -0.227278  
life_expectancy  0.384190    0.017870   -0.335278  
freedom          1.000000    0.326864   -0.487267  
generosity       0.326864    1.000000   -0.287592  
cor

(Code snippets for the PCA were borrowed and modified from [this blog post](https://medium.com/@andymdc31/using-pca-in-a-machine-learning-pipeline-b6fe3492b1b9))

In [35]:
pca = PCA()
data_processed_pca = pca.fit_transform(data_processed) 
total_explained_variance = pca.explained_variance_ratio_.cumsum()

In [36]:
# Find the number of components that explain 95% of the variance.
total_explained_variance 

array([0.53888913, 0.73394561, 0.82961535, 0.90455973, 0.95526017,
       0.98300307, 1.        ])

In [37]:
# Transform the data set with the new principal components. 
pca = PCA(n_components=5)
train_data_proc = pca.fit_transform(data_processed)
test_data_proc = pca.transform(test_data_processed)

In [38]:
# Exract the most important features for each principal component (source: Copilot)
most_important = [np.abs(pca.components_[i]).argmax() for i in range(5)]
most_important_names = [all_attribs[most_important[i]] for i in range(5)]
dictionary = {"PC1": most_important_names[0], "PC2": most_important_names[1],
                "PC3": most_important_names[2], "PC4": most_important_names[3],
                "PC5": most_important_names[4]}
importance_df = pd.DataFrame(dictionary.items(), columns=["PC", "Feature Name"])
importance_df

,PC,Feature Name
0,PC1,happiness
1,PC2,generosity
2,PC3,corruption
3,PC4,freedom
4,PC5,social_support


### IV. Analysis

The preprocessing step cannot be overlooked in any machine learning project. We noticed something odd in our initial merging of the global hunger, world happiness, and world mental health datasets: there were many NaN values. Furthermore, many of the countries did not have mental health statistics until 2006, while others started in 1990. We initially assumed this was due to a lack of sampling, and that other countries weren't included until 2006. This was a consequence of not looking close enough at our original data set. Upon further examination, we found that all of the countries in the mental health data set were reported from 1990 to 2017, so we concluded that the problem was due to our merging. 

We experimented with different types of merging that were expanded upon in class, and we found that the outer merge (our initial approach) and the left merge yielded thousands of NaN values. This was far too risky to impute because it could significantly alter models. However, the right merge only had 341 missing values, which would be very easy to impute considering the scope of the data. However, one of the disadvantages of this new merge was that we had to eschew the global hunger data set altogether. Although this was an excellent predictor to include, the data only showed rates from 2000, 2006, 2012, and 2020, so our timespan was quite limited. 

Another point of contention was how we originally dropped the NaN values. This also influenced our merge, because we did not consider that some of the country codes may have contained NaN values. Therefore, many country rows were removed from our data set, which did not give us the full scope of what we could analyze. To fix this problem, we modified our program so that the "Code" and "index" columns were dropped before we dropped the NaN values. 

We also feature engineered the mental health columns in order to obtain an overall mental health score. For each row for all of the disorder columns, we computed a score based on the quantile the rate belonged to. For instance, if a country-year pair had a schizophrenia rate of 0.21%, it would be above the 50th percentile, so we would give it a score of 3. 

After computing the scores for each disorder, we applied a function that summed up the scores for all 7 columns, and added a new column with the sum of the scores. This sum ranged from 7 to 28, so we decided to compute the quantiles again, but with categories of "Low", "Medium-Low", "Medium-High", and "High". For example, if a row had a total score of 15, it would fall in the "Medium-Low" category. 

We dropped all of the mental health columns so that the only information would be the outcome category. This feature engineering allowed us to create a more effective model, but we also ran into another problem: two of the environmental factors (GDP per capita and lifespan) were highly correlated. Therefore, we had to perform dimension reduction in the form of a PCA to get a less overfitted model. 

## Part II: Classification Analysis 

### I. Objective 

The goal of this classification analysis is to use the social and environmental predictors to determine whether a country-year pair falls into one of four mental health prevalence categories: "Low", "Medium-High", "Medium-Low", and "High". In the grand scheme of things, this classification will hopefully result in region or country-level policy-making and reforms, based on influential factors. According to the PCA feature extraction above, the strongest factors are happiness and generosity.  

### II. Classifier Implementation

In [39]:
train_data_proc

array([[-1.64889862, -1.58252625, -0.09766766, -1.07803374, -0.31161313],
       [-2.95466224,  0.56006828,  2.75357188,  1.48501091,  0.68226175],
       [-2.40306793, -0.52878851,  0.06068631, -0.11706152, -0.033193  ],
       ...,
       [ 0.54457225, -0.37512869, -0.77857619,  0.5017141 , -0.97779032],
       [-3.68593427,  1.26526815,  0.84552339,  0.54592239,  0.73176   ],
       [ 3.00760374,  1.70106515,  0.17638817, -0.68320765,  0.52898942]])

In [40]:
# Choose the best possible classifier 
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(random_state=42),
    GaussianNB()
]

In [41]:
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, train_data_proc, y_train, cv=5)
    print(f"{name}: {scores.mean()}")

Nearest Neighbors: 0.6837089917381889
Linear SVM: 0.5975321515467501


RBF SVM: 0.6902917034303895
Decision Tree: 0.5952969171947273
Random Forest: 0.620908531857437
Neural Net: 0.6537846581642203
AdaBoost: 0.512809817919307
Naive Bayes: 0.5595599048153793


**Model 1: Random Forest Classifier**

In [42]:
np.random.seed(42)
rf = RandomForestClassifier()
rf.fit(train_data_proc, y_train)
print("Default cross-val: ",cross_val_score(rf, train_data_proc, y_train, cv=5).mean())
predict = rf.predict(test_data_proc)
accuracy = rf.score(test_data_proc, y_test)
print("Default accuracy: ", accuracy)

Default cross-val:  0.685195583005802
Default accuracy:  0.6880466472303207


**Model 2: K-nearest neighbors**

In [43]:
np.random.seed(42)
kn = KNeighborsClassifier()
kn.fit(train_data_proc, y_train)
print("Default cross-val score: ",cross_val_score(kn, train_data_proc, y_train, cv=5).mean())
predict = kn.predict(test_data_proc)
accuracy = kn.score(test_data_proc, y_test)
print ("Default accuracy: ", accuracy)

Default cross-val score:  0.6837089917381889
Default accuracy:  0.6763848396501457


**Model 3: RBF Support Vector Machine**

In [44]:
np.random.seed(42)
svc = SVC(gamma=2, C=1)
svc.fit(train_data_proc, y_train)
print("Default cross-val score: ",cross_val_score(svc, train_data_proc, y_train, cv=5).mean())
predict = svc.predict(test_data_proc)
accuracy = svc.score(test_data_proc, y_test)
print ("Default accuracy: ", accuracy)

Default cross-val score:  0.6902917034303895
Default accuracy:  0.6997084548104956


### III. Hyperparameter tuning 

**Model 1: Random Forest**

In [45]:
# Grid search CV for Random Forest (testing 3 x 2 params) 
param_grid = {
                 'n_estimators': list(range(10, 100, 10)),
                 'max_depth': list(range(2, 10)),
                 'criterion': ['gini', 'entropy', 'log_loss']
             }

In [46]:
grid_results = GridSearchCV(rf, param_grid)
grid_results.fit(train_data_proc, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90]})

In [47]:
print("Best tuned parameters: ", grid_results.best_params_)
print("Best tuned score: ", grid_results.best_score_)

Best tuned parameters:  {'criterion': 'entropy', 'max_depth': 9, 'n_estimators': 40}
Best tuned score:  0.6881072698590947


In [48]:
np.random.seed(42)
rf_optimal = RandomForestClassifier(criterion="entropy", max_depth=9, n_estimators=40)
rf_optimal.fit(train_data_proc, y_train)
print("Best cross-val score (RF tuned):" ,cross_val_score\
      (rf_optimal, train_data_proc, y_train, cv=5).mean())
predict = rf_optimal.predict(test_data_proc)
accuracy = rf_optimal.score(test_data_proc, y_test)
print("Best accuracy (RF tuned):", accuracy)

Best cross-val score (RF tuned): 0.6815539691452102
Best accuracy (RF tuned): 0.685131195335277


In [49]:
# Copilot prompt: Compute precision, recall, and F1 score for the best classifier.
print(classification_report(y_test, rf_optimal.predict(test_data_proc)))

              precision    recall  f1-score   support

        High       0.70      0.77      0.73       146
         Low       0.73      0.81      0.77        75
 Medium High       0.60      0.23      0.33        39
  Medium Low       0.64      0.64      0.64        83

    accuracy                           0.69       343
   macro avg       0.67      0.61      0.62       343
weighted avg       0.68      0.69      0.67       343



*Conclusion:* The default parameters produce the highest accuracy (68.8%), so we will use these parameters over the tuned parameters. 

**Model 2: Nearest neighbors classifier**

In [50]:
kn_param_grid = {
                    'n_neighbors': list(range(2, 20)),
                    'weights': ['uniform', 'distance'],
                    'leaf_size': list(range(10, 50, 10)),
                    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
                }

In [51]:
kn_grid_results = GridSearchCV(kn, kn_param_grid)
kn_grid_results.fit(train_data_proc, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [10, 20, 30, 40],
                         'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19],
                         'weights': ['uniform', 'distance']})

In [52]:
print("Best tuned parameters: ", kn_grid_results.best_params_)
print("Best tuned score: ", kn_grid_results.best_score_)

Best tuned parameters:  {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 8, 'weights': 'distance'}
Best tuned score:  0.6946659180235824


In [53]:
kn_optimal = KNeighborsClassifier(n_neighbors=8, weights='distance', leaf_size=10)
kn_optimal.fit(train_data_proc, y_train)
print("Best cross-val score (KN tuned):",cross_val_score(kn_optimal, train_data_proc, y_train, cv=5).mean())
predict = kn_optimal.predict(test_data_proc)
accuracy = kn_optimal.score(test_data_proc, y_test)
print("Best accuracy (RF tuned): ", accuracy)

Best cross-val score (KN tuned): 0.6946659180235824
Best accuracy (RF tuned):  0.6938775510204082


In [54]:
print(classification_report(y_test, kn_optimal.predict(test_data_proc)))

              precision    recall  f1-score   support

        High       0.72      0.77      0.75       146
         Low       0.75      0.79      0.77        75
 Medium High       0.62      0.41      0.49        39
  Medium Low       0.61      0.60      0.61        83

    accuracy                           0.69       343
   macro avg       0.67      0.64      0.65       343
weighted avg       0.69      0.69      0.69       343



*Conclusion:* The best tuned parameters return a higher accuracy than the default parameters, at 69.39%. Please use the kn_optimal pipeline when running a model with this classifier. 

**Model 3: SVM**

In [55]:
svm_param_grid = {
                    'degree' : list(range(2, 10)),
                    'coef0' : list(range(0, 10)),
                    'max_iter' : list(range(1000, 3000, 100))
                }

In [56]:
svm_rbf_grid_results = GridSearchCV(svc, svm_param_grid)
svm_rbf_grid_results.fit(train_data_proc, y_train)

/Users/katewasmer/Desktop/SI_618/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(estimator=SVC(C=1, gamma=2),
             param_grid={'coef0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'degree': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_iter': [1000, 1100, 1200, 1300, 1400, 1500, 1600,
                                      1700, 1800, 1900, 2000, 2100, 2200, 2300,
                                      2400, 2500, 2600, 2700, 2800, 2900]})

In [57]:
print(svm_rbf_grid_results.best_params_)
print(svm_rbf_grid_results.best_score_)

{'coef0': 0, 'degree': 2, 'max_iter': 1000}
0.6902917034303895


In [58]:
print(classification_report(y_test, svc.predict(test_data_proc)))

              precision    recall  f1-score   support

        High       0.72      0.75      0.73       146
         Low       0.79      0.80      0.79        75
 Medium High       0.71      0.26      0.38        39
  Medium Low       0.60      0.73      0.66        83

    accuracy                           0.70       343
   macro avg       0.71      0.63      0.64       343
weighted avg       0.71      0.70      0.69       343



*Conclusion:* The grid search did not converge after 1000 iterations, so we will maintain the default parameters for the SVM at 69.97%. 

### IV. Interpretation and Results 

After tuning the hyperparameters of each of the classifiers, the best classifier for each metric has been listed below: 

- Accuracy: SVM (RBF) - 69.97% $\approx$ 70%
- Precision (macro average): SVM (RBF) - 71%
- Precision (weighted average): SVM (RBF) - 71%
- Recall (macro average): K-nearest neighbors - 64% 
- Recall (weighted average): SVM (RBF) - 70%
- F1 (macro average): K-nearest neighbors - 65% 
- F1 (weighted average): Tiebreaker between SVM (RBF) and KNN - 69%

When taking all of these metrics into account, our best bet is to use an SVM-RBF classifier with the default values. For 4/7 of these statistics, the percentage exceeds 70%. For accuracy in particular, most machine learning engineers aim for a 70-90% accuracy. (That being said, we believe that our accuracy could increase with more data and considering more data frames to merge).

However, accuracy is not the only metric that should be looked at when judging the efficacy of a machine learning model. Precision and recall are especially important. Since the samples are unbalanced (shown by the support values), we want to use the weighted averages instead of the macro averages. The weighted precision for the SVM says that when the model predicts a positive value, the percentage of true positives is 71%, while 29% are false positives. We generally want to get a 70-80% precision rate, so this supports the idea that the SVM is the best model. The weighted recall rate (or positive predictive value) is 70%. 

This is why it's important to look at a wide variety of metrics, because the accuracy for each models were nearly identical (ranging between roughly 68-70%), but there were noticeable discrepancies in the precision-recall rates. We hope that this model will be a starting point for identifying and resolving factors that negatively impact a population's mental health. 